# Importing the Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2 as cv

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Importing and Augmenting the Dataset

In [ ]:
train_datagen = ImageDataGenerator(
    brightness_range = [0.3, 0.9],
    shear_range = 0.2,
    horizontal_flip = True,
    rescale = 1./255,
validation_split = 0.2)

train_data = train_datagen.flow_from_directory(
    "Plants Classifier/Classes",
    target_size = (150, 150),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = 32,
    shuffle = True,
    subset = 'training')
validation_data = train_datagen.flow_from_directory(
    "Plants Classifier/Classes",
    target_size = (150, 150),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = 32,
    shuffle = True,
    subset = 'validation')

In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1./255)
test_data = test_datagen.flow_from_directory(
    "Plants Classifier/Classes - Test",
    target_size = (150, 150),
    color_mode = 'rgb',
    class_mode = 'binary',
    batch_size = 32,
    shuffle = False) 

# Data Exploration

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(2, 2, figsize = (20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
imgs, labels = next(train_data)
plotImages(imgs)
print(labels[:4])

In [ ]:
x = ["Train", "Test"]
height = [train_data.samples, test_data.samples]
plt.bar(x, height, width = 0.5, color = "#45BEE6", edgecolor = "k")
plt.title("Number of Plant Images", fontweight = "bold")
plt.xlabel("Data")
plt.ylabel("No. of Samples")
plt.show()

In [ ]:
X = ['Toxic', 'Non-Toxic'] 
toxic = [4000, 4000] 
non_toxic = [500, 500] 
  
X_axis = np.arange(len(X)) 
  
plt.bar(X_axis - 0.15, toxic, 0.3, edgecolor = "k", label = 'Train') 
plt.bar(X_axis + 0.15, non_toxic, 0.3, edgecolor = "k", label = 'Test') 

plt.xticks(X_axis, X) 
plt.xlabel("Classes") 
plt.ylabel("Number of Images") 
plt.title("Number of Images in each Dataset", fontweight = "bold") 
plt.legend(loc = 0) 
plt.show() 

In [ ]:
count = 1
for i in range(202, 206):
    img = cv.imread(f'Plants Classifier/Classes/Toxic/{i}.jpg')
    img = cv.resize(img, (200, 200))
    plt.subplot(2, 2, count)
    plt.imshow(img)
    plt.title('Toxic', fontweight = "bold", color = "red")
    plt.axis('off')
    count += 1
plt.show()

In [ ]:
count = 1
for i in range(102, 106):
    img = cv.imread(f'Plants Classifier/Classes/Non-Toxic/{i}.jpg')
    img = cv.resize(img, (200, 200))
    plt.subplot(2, 2, count)
    plt.imshow(img)
    plt.title('Non-Toxic', fontweight = "bold", color = "green")
    plt.axis('off')
    count += 1
plt.show()

# Building and Training the Model

In [ ]:
model = tf.keras.models.Sequential(
[
    tf.keras.layers.Conv2D(filters = 16, kernel_size = 3, input_shape = [150, 150, 3]),
    tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2),
    tf.keras.layers.Conv2D(filters = 32, kernel_size = 3),
    tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2),
    tf.keras.layers.Conv2D(filters = 64, kernel_size = 3),
    tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2),
    tf.keras.layers.Conv2D(filters = 128, kernel_size = 3),
    tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dense(128, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

In [ ]:
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
from tensorflow.keras.utils import plot_model
model.summary()

In [ ]:
plot_model(model, show_shapes = True, show_layer_names = True, dpi = 80)

In [ ]:
history1 = model.fit(x = train_data, validation_data = validation_data, epochs = 3)

## Metrics Visualization

In [ ]:
sns.set_style("darkgrid")
acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']
loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, loss, 'go-', label = 'Training Loss')
plt.plot(epochs, val_loss, 'bo-', label = 'Validation Loss')
plt.title('Loss Curve', fontweight = "bold")
plt.legend(loc = 0)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.plot(epochs, acc, 'ro-', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'bo-', label = 'Validation Accuracy')
plt.title('Accuracy', fontweight = "bold")
plt.legend(loc = 0)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
y_true = test_data.labels
y_prob = model.predict(test_data)
y_pred = []
for i in y_prob:
    if i > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
y_pred = np.array(y_pred)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
sns.set_style("white")
labels = test_data.class_indices.keys()
c_m = confusion_matrix(y_true, y_pred)

display_c_m = ConfusionMatrixDisplay(c_m, display_labels = labels)
display_c_m.plot(cmap = 'Blues')
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.title('Confusion Matrix', fontsize = 24)
plt.show()

In [ ]:
print(classification_report(y_true, y_pred))

# Fine-tuning a Pre-Trained Model (Inception V3)

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
def create_pre_trained_model(local_weights_file):
    pre_trained_model = InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = None) 
    pre_trained_model.load_weights(local_weights_file)
    
    for layer in pre_trained_model.layers:
        layer.trainable = False

    return pre_trained_model

In [ ]:
pre_trained_model = create_pre_trained_model(local_weights_file)
pre_trained_model.summary()

In [ ]:
last_desired_layer = pre_trained_model.get_layer('mixed7')
last_output = last_desired_layer.output

In [ ]:
def create_final_model(pre_trained_model, last_output):
    x = tf.keras.layers.Flatten()(last_output)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)  
    
    model = tf.keras.Model(inputs = pre_trained_model.input, outputs = x)
    model.compile(optimizer = "adam", loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:
model2 = create_final_model(pre_trained_model, last_output)
history2 = model2.fit(x = train_data, validation_data = validation_data, epochs = 5)

## Metrics Visualization

In [ ]:
sns.set_style("darkgrid")
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']
loss = history2.history['loss']
val_loss = history2.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, loss, 'ro--', label = 'Training Loss')
plt.plot(epochs, val_loss, 'bo--', label = 'Validation Loss')
plt.title('Loss Curve', fontweight = "bold")
plt.legend(loc = 0)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.plot(epochs, acc, 'ro--', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'bo--', label = 'Validation Accuracy')
plt.title('Accuracy', fontweight = "bold")
plt.legend(loc = 0)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
y_prob = model2.predict(test_data)
y_pred = []
for i in y_prob:
    if i > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)
y_pred = np.array(y_pred)

In [ ]:
sns.set_style("white")
c_m = confusion_matrix(y_true, y_pred)

labels = test_data.class_indices.keys()
display_c_m = ConfusionMatrixDisplay(c_m, display_labels = labels)
display_c_m.plot(cmap = 'Blues')
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.title('Confusion Matrix', fontsize = 24)
plt.show()

In [ ]:
print(classification_report(y_true, y_pred))